  CÓDIGO DE AUTOMATIZACIÓN DE LA BBDD DE SQM  (ITERA SOBRE SISTEMAS Y SUBSISTEMAS Y PUNTOS DE MONITOREO)

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os
import time
from selenium.common.exceptions import TimeoutException

def configure_driver(download_folder):
    options = Options()
    prefs = {
        "download.default_directory": download_folder,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    return driver

base_download_path = r"C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM"


try:
    temp_driver = webdriver.Chrome()
    temp_driver.get("https://www.sqmsenlinea.com/env-systems?type=followup")
    WebDriverWait(temp_driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "select[name='data_source_type_id']")))
    Select(temp_driver.find_element(By.CSS_SELECTOR, "select[name='data_source_type_id']")).select_by_visible_text("Niveles")
    select_sistema = Select(WebDriverWait(temp_driver, 10).until(
        EC.element_to_be_clickable((By.NAME, "environmental_system_id"))))
    sistemas = [option.text for option in select_sistema.options[1:]]
    temp_driver.quit()
    

    for sistema in sistemas:
        sistema_folder = os.path.join(base_download_path, sistema)
        if not os.path.exists(sistema_folder):
            os.makedirs(sistema_folder)
        
        driver = configure_driver(sistema_folder)
        driver.get("https://www.sqmsenlinea.com/env-systems?type=followup")
        # SELECCIONANDO NIVELES
        Select(WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "select[name='data_source_type_id']")))).select_by_visible_text("Niveles")
        Select(WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME, "environmental_system_id")))).select_by_visible_text(sistema)
        time.sleep(5)
        # SELECCIONANDO SUBSISTEMAS
        select_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "children_id")))
        select_subsistema = Select(select_element)
        time.sleep(2)
        # COMPROBANDO SI HAY SUBSISTEMAS
        if len(select_subsistema.options) > 1:
            # SI HAY SUBSISTEMAS, SE RECORREN Y SE DESCARGAN LOS PUNTOS DE MONITOREO
            for index_subsistema in range(1, len(select_subsistema.options)):
                print(f"Manejando subsistema con índice {index_subsistema}")
                select_subsistema.select_by_index(index_subsistema)
                # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "export-btn-niveles-manuales"))).click()
                
                time.sleep(3)
        #SI NO HYA SUBSISTEMAS, SE DESCARGAN LOS PUNTOS DE MONITOREO DIRECTAMENTE 
        else:
            print("No hay subsistemas. Manejando puntos de monitoreo directamente.")
            select_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, "infrastructure_id")))
            select = Select(select_element)
            # modificacion de partida
            for elements in range(1,len(select.options)): # min(3, len(select.options)) eliminar 3 y dejar solo len para la descarga de todos los puntos
                select.select_by_index(elements)
                time.sleep(5)
                try:
                    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "export-btn-niveles-manuales"))).click()
                # CLICK EN DESCARGA
                # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "export-btn-niveles-manuales"))).click()
                    time.sleep(10)
                    select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "infrastructure_id")))
                    select = Select(select_element)
                    options = select.options
                except TimeoutException:
                # VUELVE A DEJAR DISPO LA LISTA DESPLEGABLE DE PTOS 
                    select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "infrastructure_id")))
                    select = Select(select_element)
                    options = select.options
                
        driver.quit()
        

finally:
    print("¡¡¡Proceso completado...!!!")


No hay subsistemas. Manejando puntos de monitoreo directamente.
No hay subsistemas. Manejando puntos de monitoreo directamente.
No hay subsistemas. Manejando puntos de monitoreo directamente.
No hay subsistemas. Manejando puntos de monitoreo directamente.
No hay subsistemas. Manejando puntos de monitoreo directamente.
No hay subsistemas. Manejando puntos de monitoreo directamente.
Manejando subsistema con índice 1
Manejando subsistema con índice 2
Manejando subsistema con índice 3
Manejando subsistema con índice 4
Manejando subsistema con índice 5
¡¡¡Proceso completado...!!!


In [10]:
import os
import glob
import pandas as pd

def obtener_rutas_excel(ruta_principal):
    """Obtiene todas las rutas de archivos Excel en la ruta principal y sus subcarpetas."""
    rutas_excel = []
    
    # Recorre todas las carpetas y subcarpetas en la ruta principal
    for carpeta, subcarpetas, archivos in os.walk(ruta_principal):
        # Busca todos los archivos .xlsx en la carpeta actual
        archivos_excel = glob.glob(os.path.join(carpeta, '*.xlsx'))
        
        # Agrega las rutas de los archivos Excel a la lista
        rutas_excel.extend(archivos_excel)
    
    return rutas_excel

def cargar_hoja_especifica(ruta, hoja):
    """Carga una hoja específica de un archivo Excel y la devuelve como DataFrame."""
    try:
        # Lee una hoja específica del archivo Excel en un DataFrame
        df = pd.read_excel(ruta, sheet_name=hoja)
        return df
    except Exception as e:
        print(f"Error al cargar la hoja '{hoja}' del archivo {ruta}: {e}")
        return None

def cargar_y_combinar_archivos_excel(rutas_excel):
    """Carga y combina hojas específicas ('Nivel Manual' y 'Nivel Continuo') de múltiples archivos Excel."""
    dataframes_nivel_manual = []
    dataframes_nivel_continuo = []
    
    for ruta in rutas_excel:
        df_nivel_manual = cargar_hoja_especifica(ruta, 'Nivel Manual')
        if df_nivel_manual is not None:
            dataframes_nivel_manual.append(df_nivel_manual)
        
        df_nivel_continuo = cargar_hoja_especifica(ruta, 'Nivel Continuo')
        if df_nivel_continuo is not None:
            dataframes_nivel_continuo.append(df_nivel_continuo)
    
    # Combina todos los DataFrames de cada tipo en uno solo
    df_combined_nivel_manual = pd.concat(dataframes_nivel_manual, ignore_index=True) if dataframes_nivel_manual else pd.DataFrame()
    df_combined_nivel_continuo = pd.concat(dataframes_nivel_continuo, ignore_index=True) if dataframes_nivel_continuo else pd.DataFrame()
    
    return df_combined_nivel_manual, df_combined_nivel_continuo

# Especifica la ruta principal donde se encuentran las carpetas
ruta_principal = r"C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM"

# Obtener todas las rutas de archivos Excel
rutas_de_archivos_excel = obtener_rutas_excel(ruta_principal)

# Carga y combina todas las hojas "Nivel Manual" y "Nivel Continuo" en DataFrames separados
df_combined_nivel_manual, df_combined_nivel_continuo = cargar_y_combinar_archivos_excel(rutas_de_archivos_excel)

# Imprime las primeras filas de los DataFrames combinados
print("Nivel Manual:")
print(df_combined_nivel_manual.head())
print("\nNivel Continuo:")
print(df_combined_nivel_continuo.head())

# Muestra los DataFrames combinados
df_combined_nivel_manual, df_combined_nivel_continuo


Error al cargar la hoja 'Nivel Continuo' del archivo C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM\Aguas de Quelana\AGUAS DE QUELANA_D-2.xlsx: Worksheet named 'Nivel Continuo' not found
Error al cargar la hoja 'Nivel Continuo' del archivo C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM\Aguas de Quelana\AGUAS DE QUELANA_GD-02.xlsx: Worksheet named 'Nivel Continuo' not found
Error al cargar la hoja 'Nivel Continuo' del archivo C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM\Aguas de Quelana\AGUAS DE QUELANA_L13-5.xlsx: Worksheet named 'Nivel Continuo' not found
Error al cargar la hoja 'Nivel Continuo' del archivo C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM\Aguas de Quelana\AGUAS DE QUELANA_L13-6.xlsx: Worksheet named 'Nivel Continuo' not found
Error al cargar la hoja 'Nivel Continuo' del archivo C:\Users\fdona\OneDrive - Flow\Escritorio\DESCARGAS_SQM\Aguas de Quelana\AGUAS DE QUELANA_L13-7.xlsx: Worksheet named 'Nivel Continuo' not found
Error al car

(                     Fecha Compañia Punto de monitoreo Tipo nivel  \
 0      2006-02-28 14:25:00      SQM            CAMAR-2     Manual   
 1      2006-03-31 12:12:00      SQM            CAMAR-2     Manual   
 2      2006-04-30 10:45:00      SQM            CAMAR-2     Manual   
 3      2006-05-31 17:05:00      SQM            CAMAR-2     Manual   
 4      2006-06-30 15:06:00      SQM            CAMAR-2     Manual   
 ...                    ...      ...                ...        ...   
 98327  2024-04-14 12:34:00      SQM  REGLILLA TILOPOZO     Manual   
 98328  2024-04-28 10:47:00      SQM  REGLILLA TILOPOZO     Manual   
 98329  2024-05-12 11:46:00      SQM  REGLILLA TILOPOZO     Manual   
 98330  2024-05-26 12:02:00      SQM  REGLILLA TILOPOZO     Manual   
 98331  2024-06-09 12:18:00      SQM  REGLILLA TILOPOZO     Manual   
 
        Nivel (msnm) Cota Punto Referencia (msnm)  Nivel bajo referencia (m)  \
 0          2323.208                    2.379,228                     56.020  

CONVERSION A FLOAT


In [14]:

df_combined_nivel_manual['Cota Punto Referencia (msnm)'] = df_combined_nivel_manual['Cota Punto Referencia (msnm)'].str.replace('.', '').str.replace(',', '.').astype(float)

In [19]:
# campo Fecha convertir a datetime
df_combined_nivel_continuo['Fecha'] = pd.to_datetime(df_combined_nivel_continuo['Fecha'])
df_combined_nivel_continuo

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),Comentarios
0,2023-06-02 21:00:00,SQM,CAMAR-2,Continuo,2322.410,2380.057,57.647,NaN
1,2023-06-03 21:00:00,SQM,CAMAR-2,Continuo,2322.415,2380.057,57.642,NaN
2,2023-06-04 21:00:00,SQM,CAMAR-2,Continuo,2322.410,2380.057,57.647,NaN
3,2023-06-05 21:00:00,SQM,CAMAR-2,Continuo,2322.410,2380.057,57.647,NaN
4,2023-06-06 21:00:00,SQM,CAMAR-2,Continuo,2322.417,2380.057,57.640,NaN
...,...,...,...,...,...,...,...,...
126570,2024-04-12 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.167,2301.766,0.599,NaN
126571,2024-04-13 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.173,2301.766,0.593,NaN
126572,2024-04-14 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.190,2301.766,0.576,NaN
126573,2024-04-15 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.146,2301.766,0.620,NaN


FILTRAR DATAFRAMES POR FECHAS (EN DATA CONTINUO DATA HASTA 2023-10-29 )

In [21]:
df_filtered = df_combined_nivel_continuo[df_combined_nivel_continuo['Fecha'] >= '2023-10-30']
print(df_filtered)


                     Fecha Compañia   Punto de monitoreo Tipo nivel  \
150    2023-10-30 21:00:00      SQM              CAMAR-2   Continuo   
151    2023-10-31 21:00:00      SQM              CAMAR-2   Continuo   
152    2023-11-01 21:00:00      SQM              CAMAR-2   Continuo   
153    2023-11-02 21:00:00      SQM              CAMAR-2   Continuo   
154    2023-11-03 21:00:00      SQM              CAMAR-2   Continuo   
...                    ...      ...                  ...        ...   
126570 2024-04-12 20:00:00      SQM  REGLILLA PUILAR SQM   Continuo   
126571 2024-04-13 20:00:00      SQM  REGLILLA PUILAR SQM   Continuo   
126572 2024-04-14 20:00:00      SQM  REGLILLA PUILAR SQM   Continuo   
126573 2024-04-15 20:00:00      SQM  REGLILLA PUILAR SQM   Continuo   
126574 2024-04-16 20:00:00      SQM  REGLILLA PUILAR SQM   Continuo   

        Nivel (msnm)  Cota Punto Referencia (msnm)  Nivel bajo referencia (m)  \
150         2322.414                      2380.057                

( Y EN MANUAL 2023-11-06)

In [62]:
df_FILTRADO_manual = df_combined_nivel_manual[df_combined_nivel_manual['Fecha'] >= '2023-11-07']
print(df_FILTRADO_manual)

                    Fecha Compañia Punto de monitoreo Tipo nivel  \
263   2023-11-24 12:43:00      SQM            CAMAR-2     Manual   
264   2023-12-22 12:40:00      SQM            CAMAR-2     Manual   
265   2024-01-16 10:32:00    SEGEA            CAMAR-2     Manual   
266   2024-01-16 11:33:00      SQM            CAMAR-2     Manual   
267   2024-02-13 12:10:00      SQM            CAMAR-2     Manual   
...                   ...      ...                ...        ...   
98327 2024-04-14 12:34:00      SQM  REGLILLA TILOPOZO     Manual   
98328 2024-04-28 10:47:00      SQM  REGLILLA TILOPOZO     Manual   
98329 2024-05-12 11:46:00      SQM  REGLILLA TILOPOZO     Manual   
98330 2024-05-26 12:02:00      SQM  REGLILLA TILOPOZO     Manual   
98331 2024-06-09 12:18:00      SQM  REGLILLA TILOPOZO     Manual   

       Nivel (msnm)  Cota Punto Referencia (msnm)  Nivel bajo referencia (m)  \
263        2322.399                      2380.057                     57.658   
264        2322.404    

In [25]:
df_FILTRADO_manual.describe()

,Fecha,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m)
count,7011,5260.000000,7011.000000,5260.000000
mean,2024-02-26 17:59:00.059905792,2305.350155,2309.309674,6.170071
min,2023-11-07 08:28:00,2287.780000,2300.141000,0.026000
25%,2024-01-03 10:55:30,2299.104000,2301.170000,1.379000
50%,2024-02-27 10:45:00,2300.096000,2301.782000,2.260500
75%,2024-04-22 09:50:00,2310.743000,2314.614000,4.859250
max,2024-06-16 10:51:00,2333.482000,2426.055000,103.793000
std,NaN,8.679324,16.870444,12.194560


In [27]:
import pandas as pd

def generar_temporalidad(archivo_csv):
    """
    Genera un DataFrame de temporalidad con la fecha mínima, la fecha máxima y la cantidad de datos
    por 'Nombre infraestructura' a partir de un dataset de concentraciones.

    Returns:
    pd.DataFrame: DataFrame con la temporalidad
    """
    # Cargar el archivo CSV en un DataFrame
    df = archivo_csv
    
    # Convertir la columna 'Fecha' a tipo datetime
    # df['Fecha'] = pd.to_datetime(df['FechaMuestra'])
    
    # Agrupar por 'Nombre infraestructura' y calcular fecha mínima, fecha máxima y cantidad de datos
    temporalidad_df = df.groupby('Punto de monitoreo').agg(
        fecha_min=('Fecha', 'min'),
        fecha_max=('Fecha', 'max'),
        cantidad_datos=('Nivel (msnm)', 'count')
    ).reset_index()
    # temporalidad_df['fecha_min'] = temporalidad_df['fecha_min'].dt.strftime('%d %B %Y')
    # temporalidad_df['fecha_max'] = temporalidad_df['fecha_max'].dt.strftime('%d %B %Y')
    
    return temporalidad_df

# Uso de la función
archivo = 'ruta/al/archivo.csv'  # Reemplaza con la ruta a tu archivo CSV
resultado = generar_temporalidad(df_FILTRADO_manual)
print(resultado)
resultado

    Punto de monitoreo           fecha_min           fecha_max  cantidad_datos
0                 1001 2023-11-21 12:02:00 2024-05-09 15:10:00               2
1                 1024 2023-11-21 13:11:00 2024-05-09 12:37:00               1
2                 1027 2023-11-09 11:17:00 2024-06-10 13:08:00              17
3                 1028 2023-11-07 10:45:00 2024-06-15 11:48:00             240
4                 1906 2023-11-19 11:31:00 2024-06-13 11:47:00               8
..                 ...                 ...                 ...             ...
183             SOPM-5 2023-11-19 10:31:00 2024-06-13 09:25:00               8
184      SOPM-7 (L2-6) 2023-11-29 14:53:00 2024-06-13 16:05:00              12
185      SOPM-8 (L3-4) 2023-11-29 13:33:00 2024-06-15 10:54:00              18
186             SOPM-9 2023-11-29 12:23:00 2024-06-15 12:11:00               9
187            ZAR-C-S 2023-11-09 09:01:00 2024-06-12 11:18:00               8

[188 rows x 4 columns]


,Punto de monitoreo,fecha_min,fecha_max,cantidad_datos
0,1001,2023-11-21 12:02:00,2024-05-09 15:10:00,2
1,1024,2023-11-21 13:11:00,2024-05-09 12:37:00,1
2,1027,2023-11-09 11:17:00,2024-06-10 13:08:00,17
3,1028,2023-11-07 10:45:00,2024-06-15 11:48:00,240
4,1906,2023-11-19 11:31:00,2024-06-13 11:47:00,8
...,...,...,...,...
183,SOPM-5,2023-11-19 10:31:00,2024-06-13 09:25:00,8
184,SOPM-7 (L2-6),2023-11-29 14:53:00,2024-06-13 16:05:00,12
185,SOPM-8 (L3-4),2023-11-29 13:33:00,2024-06-15 10:54:00,18
186,SOPM-9,2023-11-29 12:23:00,2024-06-15 12:11:00,9


In [29]:
df_filtrado_continuo = df_filtered
df_filtrado_continuo

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),Comentarios
150,2023-10-30 21:00:00,SQM,CAMAR-2,Continuo,2322.414,2380.057,57.643,NaN
151,2023-10-31 21:00:00,SQM,CAMAR-2,Continuo,2322.412,2380.057,57.645,NaN
152,2023-11-01 21:00:00,SQM,CAMAR-2,Continuo,2322.406,2380.057,57.651,NaN
153,2023-11-02 21:00:00,SQM,CAMAR-2,Continuo,2322.412,2380.057,57.645,NaN
154,2023-11-03 21:00:00,SQM,CAMAR-2,Continuo,2322.422,2380.057,57.635,NaN
...,...,...,...,...,...,...,...,...
126570,2024-04-12 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.167,2301.766,0.599,NaN
126571,2024-04-13 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.173,2301.766,0.593,NaN
126572,2024-04-14 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.190,2301.766,0.576,NaN
126573,2024-04-15 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.146,2301.766,0.620,NaN


In [63]:
df_filtrado_manual = df_FILTRADO_manual
df_filtrado_manual

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),Comentarios
263,2023-11-24 12:43:00,SQM,CAMAR-2,Manual,2322.399,2380.057,57.658,NaN
264,2023-12-22 12:40:00,SQM,CAMAR-2,Manual,2322.404,2380.057,57.653,NaN
265,2024-01-16 10:32:00,SEGEA,CAMAR-2,Manual,2322.438,2380.057,57.619,NaN
266,2024-01-16 11:33:00,SQM,CAMAR-2,Manual,2322.420,2380.057,57.637,NaN
267,2024-02-13 12:10:00,SQM,CAMAR-2,Manual,2322.157,2380.057,57.900,NaN
...,...,...,...,...,...,...,...,...
98327,2024-04-14 12:34:00,SQM,REGLILLA TILOPOZO,Manual,2309.293,2309.549,0.256,NaN
98328,2024-04-28 10:47:00,SQM,REGLILLA TILOPOZO,Manual,2309.311,2309.549,0.238,NaN
98329,2024-05-12 11:46:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,NaN
98330,2024-05-26 12:02:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,NaN


CONECCION A LA BD

In [35]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Credenciales db
host = 'localhost'
database = 'DB_SQM-SALAR'
username = 'postgres'  
password = '1234' 

# Crea la coneccion
conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
)

engine = create_engine('postgresql://'+username+':'+password+'@'+host+'/'+database)
table_name1 = 'SQM-Niveles-Continuos'
table_name2 = 'SQM-Infraestructuras'


df_bd_continuos = pd.read_sql_table(table_name1, con=engine)#Trae la tabla de la bda un dataframe
df_infraestructuras = pd.read_sql_table(table_name2, con=engine)#



JOIN DATA UPDATE WITH DATA INFRAESTRUCTURAS

In [124]:
df_combined = pd.merge(df_filtrado_continuo, df_infraestructuras, 
                       left_on='Punto de monitoreo', 
                       right_on='HOLEID', 
                       how='left')  # Cambiar a 'left' para asegurar un left join

df_combined

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),Comentarios,HOLEID,OrigenAgua,...,TIENE_DATO_ESTRATI,SISTEMA,Alias,geometry,Latitud,Longitud,Activacion,Indicador,Sistema2,Object_ID
0,2023-10-30 21:00:00,SQM,CAMAR-2,Continuo,2322.414,2380.057,57.643,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514.0
1,2023-10-31 21:00:00,SQM,CAMAR-2,Continuo,2322.412,2380.057,57.645,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514.0
2,2023-11-01 21:00:00,SQM,CAMAR-2,Continuo,2322.406,2380.057,57.651,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514.0
3,2023-11-02 21:00:00,SQM,CAMAR-2,Continuo,2322.412,2380.057,57.645,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514.0
4,2023-11-03 21:00:00,SQM,CAMAR-2,Continuo,2322.422,2380.057,57.635,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6501,2024-04-12 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.167,2301.766,0.599,NaN,REGLILLA PUILAR SQM,Agua Superficial,...,NO,Soncor,None,POINT (587947.4399999999 7422497.03),-23.304620,-68.139902,None,None,None,445.0
6502,2024-04-13 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.173,2301.766,0.593,NaN,REGLILLA PUILAR SQM,Agua Superficial,...,NO,Soncor,None,POINT (587947.4399999999 7422497.03),-23.304620,-68.139902,None,None,None,445.0
6503,2024-04-14 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.190,2301.766,0.576,NaN,REGLILLA PUILAR SQM,Agua Superficial,...,NO,Soncor,None,POINT (587947.4399999999 7422497.03),-23.304620,-68.139902,None,None,None,445.0
6504,2024-04-15 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.146,2301.766,0.620,NaN,REGLILLA PUILAR SQM,Agua Superficial,...,NO,Soncor,None,POINT (587947.4399999999 7422497.03),-23.304620,-68.139902,None,None,None,445.0


In [126]:
df_bd_continuos.columns
hoy = pd.to_datetime('today')

# Agregando la columna 'Fecha_Update' con la fecha de hoy
df_combined['Fecha_Update'] = hoy
df_combined.columns

Index(['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
       'Cota Punto Referencia (msnm)', 'Nivel bajo referencia (m)',
       'Comentarios', 'HOLEID', 'OrigenAgua', 'Fuente', 'Nombre_Equivalencia',
       'X_UtmEste', 'Y_UtmNorte', 'CotaRef_msnm', 'DATUM',
       'MIN_FechaCoordenadaNivel', 'MAX_FechaCoordenadaNivel',
       'RanuraMin_From_m', 'RanuraMax_To_m', 'ProfundidadTotal_m',
       'TIENE_DATO_NIVEL', 'TIENE_DATO_PH', 'TIENE_DATO_QCA',
       'TIENE_DATO_ESTRATI', 'SISTEMA', 'Alias', 'geometry', 'Latitud',
       'Longitud', 'Activacion', 'Indicador', 'Sistema2', 'Object_ID',
       'Fecha_Update'],
      dtype='object')

EXTRAER COLUMNAS NECESARIAS EN FORMATO BD

In [127]:
columns_to_extract = ['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
                      'Cota Punto Referencia (msnm)', 'Nivel bajo referencia (m)',
                      'SISTEMA', 'SISTEMA', 'Fecha_Update', 'Fuente', 'Object_ID']

# Extrayendo las columnas seleccionadas
df_extracted_continuos= df_combined[columns_to_extract]

# Mostrando las primeras filas del dataframe resultante para verificar
df_extracted_continuos

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),SISTEMA,SISTEMA,Fecha_Update,Fuente,Object_ID
0,2023-10-30 21:00:00,SQM,CAMAR-2,Continuo,2322.414,2380.057,57.643,None,None,2024-06-18 18:32:46.392255,Dalannais (1979) / Re interpretacion Borde Ori...,514.0
1,2023-10-31 21:00:00,SQM,CAMAR-2,Continuo,2322.412,2380.057,57.645,None,None,2024-06-18 18:32:46.392255,Dalannais (1979) / Re interpretacion Borde Ori...,514.0
2,2023-11-01 21:00:00,SQM,CAMAR-2,Continuo,2322.406,2380.057,57.651,None,None,2024-06-18 18:32:46.392255,Dalannais (1979) / Re interpretacion Borde Ori...,514.0
3,2023-11-02 21:00:00,SQM,CAMAR-2,Continuo,2322.412,2380.057,57.645,None,None,2024-06-18 18:32:46.392255,Dalannais (1979) / Re interpretacion Borde Ori...,514.0
4,2023-11-03 21:00:00,SQM,CAMAR-2,Continuo,2322.422,2380.057,57.635,None,None,2024-06-18 18:32:46.392255,Dalannais (1979) / Re interpretacion Borde Ori...,514.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6501,2024-04-12 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.167,2301.766,0.599,Soncor,Soncor,2024-06-18 18:32:46.392255,PSAH,445.0
6502,2024-04-13 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.173,2301.766,0.593,Soncor,Soncor,2024-06-18 18:32:46.392255,PSAH,445.0
6503,2024-04-14 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.190,2301.766,0.576,Soncor,Soncor,2024-06-18 18:32:46.392255,PSAH,445.0
6504,2024-04-15 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.146,2301.766,0.620,Soncor,Soncor,2024-06-18 18:32:46.392255,PSAH,445.0


RENOMBRAR COLUMNAS

In [47]:
new_column_names = ['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
                    'Cota Punto Referencia (msnm)', 'Nivel bajo referencia (m)',
                    'Sub-Sistema', 'Sistema', 'Fecha_Update', 'Fuente', 'Object_ID']

# Renombrando las columnas
df_extracted_continuos.columns = new_column_names

In [48]:
df_concat = pd.concat([df_bd_continuos, df_extracted_continuos], ignore_index=True)

In [49]:
df_concat['Fecha'] = pd.to_datetime(df_concat['Fecha'])

DF LISTO PARA CARGAR NIVEL CONTINUO

In [84]:
df_update_Continuos = df_concat
df_update_Continuos

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),Sub-Sistema,Sistema,Fecha_Update,Fuente,Object_ID
0,2023-06-02 21:00:00,SQM,Camar-2,Continuo,2322.41000,2380.057,57.647,Aguas de Quelana,Aguas de Quelana,2023-10-30 00:00:00.000000,Nivel Continuo,469
1,2023-06-03 21:00:00,SQM,Camar-2,Continuo,2322.41500,2380.057,57.642,Aguas de Quelana,Aguas de Quelana,2023-10-30 00:00:00.000000,Nivel Continuo,469
2,2023-06-04 21:00:00,SQM,Camar-2,Continuo,2322.41000,2380.057,57.647,Aguas de Quelana,Aguas de Quelana,2023-10-30 00:00:00.000000,Nivel Continuo,469
3,2007-09-15 20:00:00,SQM,L13-1,Continuo,2315.88757,2319.445,3.557,Aguas de Quelana,Aguas de Quelana,2023-10-30 00:00:00.000000,Nivel Continuo,3
4,2007-09-16 20:00:00,SQM,L13-1,Continuo,2315.88835,2319.445,3.557,Aguas de Quelana,Aguas de Quelana,2023-10-30 00:00:00.000000,Nivel Continuo,3
...,...,...,...,...,...,...,...,...,...,...,...,...
138895,2024-04-12 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.16700,2301.766,0.599,Soncor,Soncor,2024-06-18 14:03:41.037451,PSAH,445
138896,2024-04-13 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.17300,2301.766,0.593,Soncor,Soncor,2024-06-18 14:03:41.037451,PSAH,445
138897,2024-04-14 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.19000,2301.766,0.576,Soncor,Soncor,2024-06-18 14:03:41.037451,PSAH,445
138898,2024-04-15 20:00:00,SQM,REGLILLA PUILAR SQM,Continuo,2301.14600,2301.766,0.620,Soncor,Soncor,2024-06-18 14:03:41.037451,PSAH,445


In [118]:
df_update_Continuos.to_sql(table_name1, con=engine, if_exists='replace', index=False)


900

Generar excel 

In [120]:
df_update_Continuos.to_excel('SQM-Niveles.continuos.xlsx', index=False, sheet_name='Niveles Continuos')

<H1>ACTUALIZACION NIVELES MANUALES</H1>

DF NIVEL MANUAL BD

In [72]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

# Credenciales db
host = 'localhost'
database = 'DB_SQM-SALAR'
username = 'postgres'  
password = '1234' 

# Crea la coneccion
conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
)

engine = create_engine('postgresql://'+username+':'+password+'@'+host+'/'+database)
table_name1 = 'SQM-Niveles-Continuos'
table_name2 = 'SQM-Infraestructuras'
table_name3 = 'SQM-Niveles-Manuales'


df_bd_continuos = pd.read_sql_table(table_name1, con=engine)#Trae la tabla de la bda un dataframe
df_bd_manual = pd.read_sql_table(table_name3, con=engine)#Trae la tabla de la bda un dataframe
df_infraestructuras = pd.read_sql_table(table_name2, con=engine)#



In [79]:
# Convertir la columna 'Columna_Object' a texto
df_filtrado_manual['Punto de monitoreo'] = df_filtrado_manual['Punto de monitoreo'].astype(str)


df_filtrado_manual

C:\Users\fdona\AppData\Local\Temp\ipykernel_28064\1320503225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_manual['Punto de monitoreo'] = df_filtrado_manual['Punto de monitoreo'].astype(str)


,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),Comentarios
263,2023-11-24 12:43:00,SQM,CAMAR-2,Manual,2322.399,2380.057,57.658,NaN
264,2023-12-22 12:40:00,SQM,CAMAR-2,Manual,2322.404,2380.057,57.653,NaN
265,2024-01-16 10:32:00,SEGEA,CAMAR-2,Manual,2322.438,2380.057,57.619,NaN
266,2024-01-16 11:33:00,SQM,CAMAR-2,Manual,2322.420,2380.057,57.637,NaN
267,2024-02-13 12:10:00,SQM,CAMAR-2,Manual,2322.157,2380.057,57.900,NaN
...,...,...,...,...,...,...,...,...
98327,2024-04-14 12:34:00,SQM,REGLILLA TILOPOZO,Manual,2309.293,2309.549,0.256,NaN
98328,2024-04-28 10:47:00,SQM,REGLILLA TILOPOZO,Manual,2309.311,2309.549,0.238,NaN
98329,2024-05-12 11:46:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,NaN
98330,2024-05-26 12:02:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,NaN


In [88]:
df_combined_manual = pd.merge(df_filtrado_manual, df_infraestructuras, 
                       left_on='Punto de monitoreo', 
                       right_on='HOLEID', 
                       how='inner')  # Cambiar a 'left' para asegurar un left join

df_combined_manual

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),Comentarios,HOLEID,OrigenAgua,...,TIENE_DATO_ESTRATI,SISTEMA,Alias,geometry,Latitud,Longitud,Activacion,Indicador,Sistema2,Object_ID
0,2023-11-24 12:43:00,SQM,CAMAR-2,Manual,2322.399,2380.057,57.658,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514
1,2023-12-22 12:40:00,SQM,CAMAR-2,Manual,2322.404,2380.057,57.653,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514
2,2024-01-16 10:32:00,SEGEA,CAMAR-2,Manual,2322.438,2380.057,57.619,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514
3,2024-01-16 11:33:00,SQM,CAMAR-2,Manual,2322.420,2380.057,57.637,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514
4,2024-02-13 12:10:00,SQM,CAMAR-2,Manual,2322.157,2380.057,57.900,NaN,CAMAR-2,Agua Subterránea,...,SI,None,None,POINT (598843.87 7409618.41),-23.420316,-68.032506,None,None,None,514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6991,2024-04-14 12:34:00,SQM,REGLILLA TILOPOZO,Manual,2309.293,2309.549,0.256,NaN,REGLILLA TILOPOZO,Agua Subterránea,...,None,Tilopozo,None,POINT (-68.23679855444405 -23.78503389116242),-23.785034,-68.236799,None,None,None,448
6992,2024-04-28 10:47:00,SQM,REGLILLA TILOPOZO,Manual,2309.311,2309.549,0.238,NaN,REGLILLA TILOPOZO,Agua Subterránea,...,None,Tilopozo,None,POINT (-68.23679855444405 -23.78503389116242),-23.785034,-68.236799,None,None,None,448
6993,2024-05-12 11:46:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,NaN,REGLILLA TILOPOZO,Agua Subterránea,...,None,Tilopozo,None,POINT (-68.23679855444405 -23.78503389116242),-23.785034,-68.236799,None,None,None,448
6994,2024-05-26 12:02:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,NaN,REGLILLA TILOPOZO,Agua Subterránea,...,None,Tilopozo,None,POINT (-68.23679855444405 -23.78503389116242),-23.785034,-68.236799,None,None,None,448


AÑADIR COLUMNA FECHA UPDATE 

In [91]:
hoy = pd.to_datetime('today')

# Agregando la columna 'Fecha_Update' con la fecha de hoy
df_combined_manual['Fecha_Update'] = hoy
df_combined_manual.columns

Index(['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
       'Cota Punto Referencia (msnm)', 'Nivel bajo referencia (m)',
       'Comentarios', 'HOLEID', 'OrigenAgua', 'Fuente', 'Nombre_Equivalencia',
       'X_UtmEste', 'Y_UtmNorte', 'CotaRef_msnm', 'DATUM',
       'MIN_FechaCoordenadaNivel', 'MAX_FechaCoordenadaNivel',
       'RanuraMin_From_m', 'RanuraMax_To_m', 'ProfundidadTotal_m',
       'TIENE_DATO_NIVEL', 'TIENE_DATO_PH', 'TIENE_DATO_QCA',
       'TIENE_DATO_ESTRATI', 'SISTEMA', 'Alias', 'geometry', 'Latitud',
       'Longitud', 'Activacion', 'Indicador', 'Sistema2', 'Object_ID',
       'Fecha_Update'],
      dtype='object')

In [106]:
df_bd_manual.columns
# df_combined_manual.columns

Index(['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
       'Cota Referencia(msnm)', 'Prof Nivel (m)', 'Sub-Sistema', 'Sistema',
       'Object_ID', 'Fecha_Update'],
      dtype='object')

EXTRAER DEL JOIN LAS COLUMNAS NECESARIAS

In [108]:
columns_to_extract_manual = ['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
       'Cota Punto Referencia (msnm)', 'Nivel bajo referencia (m)', 'SISTEMA', 'SISTEMA',
       'Object_ID', 'Fecha_Update']

# Extrayendo las columnas seleccionadas
df_extracted_manual= df_combined_manual[columns_to_extract_manual]


In [109]:
df_extracted_manual

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Punto Referencia (msnm),Nivel bajo referencia (m),SISTEMA,SISTEMA,Object_ID,Fecha_Update
0,2023-11-24 12:43:00,SQM,CAMAR-2,Manual,2322.399,2380.057,57.658,None,None,514,2024-06-18 14:55:10.574107
1,2023-12-22 12:40:00,SQM,CAMAR-2,Manual,2322.404,2380.057,57.653,None,None,514,2024-06-18 14:55:10.574107
2,2024-01-16 10:32:00,SEGEA,CAMAR-2,Manual,2322.438,2380.057,57.619,None,None,514,2024-06-18 14:55:10.574107
3,2024-01-16 11:33:00,SQM,CAMAR-2,Manual,2322.420,2380.057,57.637,None,None,514,2024-06-18 14:55:10.574107
4,2024-02-13 12:10:00,SQM,CAMAR-2,Manual,2322.157,2380.057,57.900,None,None,514,2024-06-18 14:55:10.574107
...,...,...,...,...,...,...,...,...,...,...,...
6991,2024-04-14 12:34:00,SQM,REGLILLA TILOPOZO,Manual,2309.293,2309.549,0.256,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
6992,2024-04-28 10:47:00,SQM,REGLILLA TILOPOZO,Manual,2309.311,2309.549,0.238,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
6993,2024-05-12 11:46:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
6994,2024-05-26 12:02:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107


RENOMBRAR COLUMNAS PARA QUE COINCIDAN CON EL FORMATO DE LA BD

In [110]:
new_column_names = ['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
       'Cota Referencia(msnm)', 'Prof Nivel (m)', 'Sub-Sistema', 'Sistema',
       'Object_ID', 'Fecha_Update']
# Renombrando las columnas
df_extracted_manual.columns = new_column_names

In [111]:
df_extracted_manual

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Referencia(msnm),Prof Nivel (m),Sub-Sistema,Sistema,Object_ID,Fecha_Update
0,2023-11-24 12:43:00,SQM,CAMAR-2,Manual,2322.399,2380.057,57.658,None,None,514,2024-06-18 14:55:10.574107
1,2023-12-22 12:40:00,SQM,CAMAR-2,Manual,2322.404,2380.057,57.653,None,None,514,2024-06-18 14:55:10.574107
2,2024-01-16 10:32:00,SEGEA,CAMAR-2,Manual,2322.438,2380.057,57.619,None,None,514,2024-06-18 14:55:10.574107
3,2024-01-16 11:33:00,SQM,CAMAR-2,Manual,2322.420,2380.057,57.637,None,None,514,2024-06-18 14:55:10.574107
4,2024-02-13 12:10:00,SQM,CAMAR-2,Manual,2322.157,2380.057,57.900,None,None,514,2024-06-18 14:55:10.574107
...,...,...,...,...,...,...,...,...,...,...,...
6991,2024-04-14 12:34:00,SQM,REGLILLA TILOPOZO,Manual,2309.293,2309.549,0.256,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
6992,2024-04-28 10:47:00,SQM,REGLILLA TILOPOZO,Manual,2309.311,2309.549,0.238,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
6993,2024-05-12 11:46:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
6994,2024-05-26 12:02:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107


UNIR DATA NUEVA A LA EXISTENTE EN BD

In [112]:
df_bd_manual

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Referencia(msnm),Prof Nivel (m),Sub-Sistema,Sistema,Object_ID,Fecha_Update
0,2007-09-04,SQM,1001,Manual,2299.740,2301.100,1.360,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
1,2007-10-02,SQM,1001,Manual,2299.746,2301.100,1.354,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
2,2007-10-27,SQM,1001,Manual,2299.743,2301.100,1.357,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
3,2007-12-06,SQM,1001,Manual,2299.740,2301.100,1.360,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
4,2007-12-27,SQM,1001,Manual,2299.730,2301.100,1.370,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
...,...,...,...,...,...,...,...,...,...,...,...
85461,2023-01-21,SQM,ZAR-C-S,Manual,2295.495,2303.141,7.646,Nucleo Salar de Atacama,Nucleo Salar de Atacama,524,NaT
85462,2023-02-28,SQM,ZAR-C-S,Manual,2295.481,2303.141,7.660,Nucleo Salar de Atacama,Nucleo Salar de Atacama,524,NaT
85463,2023-03-29,SQM,ZAR-C-S,Manual,2295.475,2303.141,7.666,Nucleo Salar de Atacama,Nucleo Salar de Atacama,524,NaT
85464,2023-04-30,SQM,ZAR-C-S,Manual,2295.466,2303.141,7.675,Nucleo Salar de Atacama,Nucleo Salar de Atacama,524,NaT


UNIR LA NUEVA DATA CON LA DATA DE LA BBDD

In [113]:
df_concat_manual = pd.concat([df_bd_manual, df_extracted_manual], ignore_index=True)

FECAH A DATETIME

In [114]:
df_concat_manual['Fecha'] = pd.to_datetime(df_concat_manual['Fecha'])

DF LISTO PARA CARGAR NIVEL MANUAL

In [115]:
df_update_Manual = df_concat_manual
df_update_Manual

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Referencia(msnm),Prof Nivel (m),Sub-Sistema,Sistema,Object_ID,Fecha_Update
0,2007-09-04 00:00:00,SQM,1001,Manual,2299.740,2301.100,1.360,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
1,2007-10-02 00:00:00,SQM,1001,Manual,2299.746,2301.100,1.354,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
2,2007-10-27 00:00:00,SQM,1001,Manual,2299.743,2301.100,1.357,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
3,2007-12-06 00:00:00,SQM,1001,Manual,2299.740,2301.100,1.360,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
4,2007-12-27 00:00:00,SQM,1001,Manual,2299.730,2301.100,1.370,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,NaT
...,...,...,...,...,...,...,...,...,...,...,...
92457,2024-04-14 12:34:00,SQM,REGLILLA TILOPOZO,Manual,2309.293,2309.549,0.256,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
92458,2024-04-28 10:47:00,SQM,REGLILLA TILOPOZO,Manual,2309.311,2309.549,0.238,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
92459,2024-05-12 11:46:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107
92460,2024-05-26 12:02:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 14:55:10.574107


In [116]:
hoy = pd.to_datetime('today')

# Agregando la columna 'Fecha_Update' con la fecha de hoy
df_update_Manual['Fecha_Update'] = hoy
df_update_Manual.columns

Index(['Fecha', 'Compañia', 'Punto de monitoreo', 'Tipo nivel', 'Nivel (msnm)',
       'Cota Referencia(msnm)', 'Prof Nivel (m)', 'Sub-Sistema', 'Sistema',
       'Object_ID', 'Fecha_Update'],
      dtype='object')

In [117]:
df_update_Manual

,Fecha,Compañia,Punto de monitoreo,Tipo nivel,Nivel (msnm),Cota Referencia(msnm),Prof Nivel (m),Sub-Sistema,Sistema,Object_ID,Fecha_Update
0,2007-09-04 00:00:00,SQM,1001,Manual,2299.740,2301.100,1.360,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,2024-06-18 16:07:17.371471
1,2007-10-02 00:00:00,SQM,1001,Manual,2299.746,2301.100,1.354,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,2024-06-18 16:07:17.371471
2,2007-10-27 00:00:00,SQM,1001,Manual,2299.743,2301.100,1.357,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,2024-06-18 16:07:17.371471
3,2007-12-06 00:00:00,SQM,1001,Manual,2299.740,2301.100,1.360,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,2024-06-18 16:07:17.371471
4,2007-12-27 00:00:00,SQM,1001,Manual,2299.730,2301.100,1.370,Nucleo Salar de Atacama,Nucleo Salar de Atacama,474,2024-06-18 16:07:17.371471
...,...,...,...,...,...,...,...,...,...,...,...
92457,2024-04-14 12:34:00,SQM,REGLILLA TILOPOZO,Manual,2309.293,2309.549,0.256,Tilopozo,Tilopozo,448,2024-06-18 16:07:17.371471
92458,2024-04-28 10:47:00,SQM,REGLILLA TILOPOZO,Manual,2309.311,2309.549,0.238,Tilopozo,Tilopozo,448,2024-06-18 16:07:17.371471
92459,2024-05-12 11:46:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 16:07:17.371471
92460,2024-05-26 12:02:00,SQM,REGLILLA TILOPOZO,Manual,2309.351,2309.549,0.198,Tilopozo,Tilopozo,448,2024-06-18 16:07:17.371471


ACTUALIZAR EN LA BD

In [119]:
df_update_Manual.to_sql(table_name3, con=engine, if_exists='replace', index=False)

462

In [121]:
df_update_Manual.to_excel('SQM-Niveles-Manual.xlsx', index=False, sheet_name='Niveles Manual')